In [1]:
import pandas as pd
import numpy as np
import math
import time

In [ ]:
#with open("C:\\Users\\Nino\\ADM HW4\\passwords1.txt", 'r') as file :
    #df = pd.DataFrame(file.read().split("\n"), columns = ["pass1"])

In [ ]:
#with open("C:\\Users\\Nino\\ADM HW4\\passwords2.txt", 'r') as file :
    #df["pass2"] = file.read().split("\n")

In [ ]:
#len(df["pass1"])

In [ ]:
#len(df["pass2"])

for implementing bloom filter, first we need to difine indipendent hash functions. Each hash function should get a string(passwords here) as number and return an integer. We decide to define 2 hash functions and build the bloom filter based on them. In the first hash function we get the ASCII code of password by multiplying the ASCII code of each character with 37 power the position of the character. Then to get indexes from these hash numbers we calculate the modular value over the first prime number after the lenght of our file(100000000).  

In [2]:
def hash_fun1 (string):
    hashdict = dict()
    h = [ord(i) for i in string]
    hashnumber = sum([(37**h.index(i))*i for i in h])% 100000007
    hashdict[string] = hashnumber
    return(hashdict)
    
    

In [3]:
hash_fun1('b*ZB/egd=!XcvV>K(qY@')['b*ZB/egd=!XcvV>K(qY@']

33987287

In the first hash function we get the ASCII code of each password by multiplying the ASCII code of each character with 37 power the difference of  character's position and 20. To get indexes from this number, we calculate the modular value over the first prime number after 100000000

In [4]:
def hash_fun2 (string):
    hashdict = dict()
    h = [ord(i) for i in string]
    hashnumber = sum([(37**(20 - h.index(i)))*i for i in h])% 100000007
    hashdict[string] = hashnumber
    return(hashdict)

In [5]:
hash_fun2('b*ZB/egd=!XcvV>K(qY@')

{'b*ZB/egd=!XcvV>K(qY@': 75648607}

The third hash function concatenate ASCII codes of each 4 characters of the password, then calculate the sum of that value and at last return the modualar value over the first prime number after the lenght of our file

In [6]:
def hash_fun3 (string) :
    hashdict = dict()
    h = [ord(i) for i in string]
    hashnumber = 0
    for i in range(0,5):
        s="".join(map(str, h[4*i : 4*i + 4]))
        try :
            hashnumber += int(s)
        except :
            pass
        hashnumber = hashnumber % 100000007
        hashdict[string] = hashnumber
        
    return(hashdict)    

In [7]:
hash_fun3('b*ZB/egd=!XcvV>K(qY@')

{'b*ZB/egd=!XcvV>K(qY@': 80872965}

Now, it's time to implement our bloom filter. we take an array of size m = 400,000,000 with all elements equal to 0, and for each password in password1 file we update the array

In [12]:
n = 100000000
m = n * 4
bfilter = np.zeros(m,int)

In [13]:
def update_filter(array) :
    with open("passwords1.txt", 'r') as file :
            for line in file :
                array[hash_fun1(line)[line]] = 1
                array[hash_fun2(line)[line]] = 1
                array[hash_fun3(line)[line]] = 1
    np.savetxt("bloomfilter.csv", array , fmt='%10d', delimiter=",")

In [14]:
def search_filter(array) :
    counter = 0
    with open("passwords2.txt", 'r') as file :
        for line in file :
            if(array[hash_fun1(line)[line]] == 1  and  array[hash_fun2(line)[line]] == 1
              and array[hash_fun3(line)[line]] == 1) :
                counter += 1
    return(counter)       

In [15]:
def bloom_filter(array):
    start = time.time()
    update_filter(array)
    duplicates = search_filter(array)
    end = time.time()
    runtime = end - start
    return(duplicates,runtime)
    

In [16]:
duplicates,runtime = bloom_filter(bfilter)

In [20]:
k = 3 #number of hash functions
n = 100000000 #elemnts in bloom filter
m = n*4 #size of bit array
p = pow(1 - math.exp(-k / (m / n)), k) #probability of false positive
p

0.14689159766038104

In [21]:
print('Number of hash function used: ', k)
print('Number of duplicates detected: ', duplicates)
print('Probability of false positives: ', p)
print('Execution time: ', runtime)

Number of hash function used:  3
Number of duplicates detected:  35450858
Probability of false positives:  0.14689159766038104
Execution time:  5766.666255474091
